# Corex LDA Error Analysis

In [2]:
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt
import pandas as pd
import re

import corextopic.corextopic as ct
import corextopic.vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [7]:
model_data_test = pd.read_csv('LDA_test.csv')

In [8]:
model_data_test

,Unnamed: 0.2,Unnamed: 0.1,index,Unnamed: 0,bookId,title,series,author,rating,description,...,university,relationships,break ups,divorce,weddings,death,family,friendship,labeled?,Contains True?
0,0,0,0,"39,822",34838660-not-part-of-the-plan,Not Part of the Plan,Blue Moon #4,Lucy Score (Goodreads Author),4.46,From the Wall Street Journal and #1 Amazon bes...,...,False,True,False,False,False,False,False,False,Yes,1.0
1,1,1,1,"34,235",20176552-dragon-age-volume-1,"Dragon Age, Volume 1",Dragon Age Graphic Novels #1-3,"David Gaider, Chad Hardin (Illustrator), Antho...",4.26,Helping set the stage for BioWare's hotly anti...,...,False,False,False,False,False,False,False,False,Yes,0.0
2,2,2,2,"27,904",124110.Dangerous_to_Know,Dangerous to Know,NaN,Barbara Taylor Bradford (Goodreads Author),3.73,"Sebastian Locke, the fifty-six-year-old patria...",...,False,True,False,True,False,True,True,False,Yes,1.0
3,3,3,3,"10,515",1046450.The_Wheel_of_Fortune,The Wheel of Fortune,NaN,Susan Howatch,4.11,"""Take me back to Oxmoon, the lost paradise of ...",...,False,True,False,False,False,False,True,False,Yes,1.0
4,4,4,4,935,872333.Blue_Bloods,Blue Bloods,Blue Bloods #1,Melissa de la Cruz (Goodreads Author),3.69,"When the Mayflower set sail in 1620, it carrie...",...,False,False,False,False,False,False,False,False,Yes,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,995,"17,361",588326.The_Blue_Helmet,The Blue Helmet,NaN,William Bell,3.42,Lee wants to be a Tarantula – a member of the ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1.0
996,996,996,996,"9,029",93007.The_Merry_Adventures_of_Robin_Hood,The Merry Adventures of Robin Hood,NaN,Howard Pyle,4.07,The Merry Adventures of Robin Hood of Great Re...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
997,997,997,997,"32,216",1085376.Before_You_Sleep,Before You Sleep,NaN,"Linn Ullmann, Tiina Nunnally (Translator)",3.34,Moving from present-day Oslo to Brooklyn in th...,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1.0
998,998,998,998,"1,036",28195.Inkspell,Inkspell,Inkworld #2,"Cornelia Funke (Goodreads Author), Anthea Bell...",3.91,"The captivating sequel to INKHEART, the critic...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
y_true = pd.read_csv('corex_y_true.csv')
y_pred = pd.read_csv('corex_bools_anchor_strength_6.csv')

In [5]:
y_true

,Unnamed: 0,university,relationships,break ups,divorce,wedding,death,family,friendship
0,0,False,True,False,False,False,False,False,False
1,1,False,False,False,False,False,False,False,False
2,2,False,True,False,True,False,True,True,False
3,3,False,True,False,False,False,False,True,False
4,4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
995,995,True,True,True,True,True,True,True,True
996,996,True,True,True,True,True,True,True,True
997,997,True,True,True,True,True,True,True,True
998,998,True,True,True,True,True,True,True,True


In [6]:
y_pred

,Unnamed: 0,university,relationships,break ups,divorce,wedding,death,family,friendship
0,0,False,True,False,True,True,False,True,True
1,1,False,False,False,True,False,False,False,False
2,2,False,False,False,True,False,True,True,False
3,3,False,True,False,False,False,False,True,False
4,4,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...
995,995,False,False,False,False,True,True,True,True
996,996,True,True,True,False,False,False,False,False
997,997,False,False,False,False,False,True,True,False
998,998,False,False,False,True,True,False,False,False
